In [1]:
import pandas as pd
import numpy as np
import gc
from scipy.stats import pearsonr
from concurrent.futures import ThreadPoolExecutor
from sklearn.feature_selection import mutual_info_classif

# 读取特征和标签

In [2]:
total_factor = pd.read_pickle('/home/datamake117/data/haris/dataset/total_factor.pkl')
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,2216,2217,2218,2219,2220,label,qcut,weight,qid,amount
0,2020-01-02,000001,0.161906,0.715487,0.093825,2.500486,1.495386,3.687422,1.265125,2.488541,...,10500.0,70.0,1.0,0.100181,0.0,0.010385,0,0,0,1.886999e+07
1,2020-01-02,000002,0.233131,0.554393,-0.052147,0.148151,0.797523,-4.361952,0.735354,0.153694,...,10564.0,67.0,1.0,0.100095,0.0,-0.044313,0,0,0,2.000000e+07
2,2020-01-02,000004,0.129032,0.902306,-0.089413,-3.294483,0.073779,-4.185775,0.243243,-3.187878,...,10579.0,61.0,0.0,0.100044,0.0,-0.034692,0,0,0,2.831250e+04
3,2020-01-02,000005,0.003936,0.962999,0.011201,-2.758275,1.244934,-4.228945,0.402985,-2.640067,...,10605.0,62.0,0.0,0.100649,0.0,-0.008585,0,0,0,1.079170e+05
4,2020-01-02,000006,0.059158,0.808410,-0.066815,0.230044,0.572983,-3.184859,0.476190,0.285741,...,10110.0,62.0,1.0,0.099624,0.0,-0.012239,0,0,0,2.818170e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398928,2024-12-31,688799,0.427302,0.000000,0.000000,0.868741,0.000000,-0.333084,0.000000,0.879594,...,1267.0,62.0,1.0,0.199906,0.0,0.047214,0,0,0,0.000000e+00
5398929,2024-12-31,688800,0.509672,0.568677,-0.055477,1.321744,0.674025,3.325044,1.700000,1.251481,...,1258.0,66.0,1.0,0.200036,0.0,0.040934,0,0,0,2.911135e+06
5398930,2024-12-31,688819,0.020425,0.971780,-0.017278,-0.562102,0.300689,0.980164,1.291667,-0.386845,...,1443.0,61.0,1.0,0.200000,0.0,0.032876,0,0,0,4.065375e+04
5398931,2024-12-31,688981,0.103586,0.720491,-0.165714,-2.509371,0.283653,0.702663,1.041734,-2.383492,...,1629.0,52.0,1.0,0.199979,0.0,-0.002054,0,0,0,2.000000e+07


In [3]:
temp = total_factor[total_factor['date'] < pd.to_datetime("2024-10-01")]
del total_factor
total_factor = temp
del temp
gc.collect()
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,2216,2217,2218,2219,2220,label,qcut,weight,qid,amount
0,2020-01-02,000001,0.161906,0.715487,0.093825,2.500486,1.495386,3.687422,1.265125,2.488541,...,10500.0,70.0,1.0,0.100181,0.0,0.010385,0,0,0,1.886999e+07
1,2020-01-02,000002,0.233131,0.554393,-0.052147,0.148151,0.797523,-4.361952,0.735354,0.153694,...,10564.0,67.0,1.0,0.100095,0.0,-0.044313,0,0,0,2.000000e+07
2,2020-01-02,000004,0.129032,0.902306,-0.089413,-3.294483,0.073779,-4.185775,0.243243,-3.187878,...,10579.0,61.0,0.0,0.100044,0.0,-0.034692,0,0,0,2.831250e+04
3,2020-01-02,000005,0.003936,0.962999,0.011201,-2.758275,1.244934,-4.228945,0.402985,-2.640067,...,10605.0,62.0,0.0,0.100649,0.0,-0.008585,0,0,0,1.079170e+05
4,2020-01-02,000006,0.059158,0.808410,-0.066815,0.230044,0.572983,-3.184859,0.476190,0.285741,...,10110.0,62.0,1.0,0.099624,0.0,-0.012239,0,0,0,2.818170e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095032,2024-09-30,688799,0.022639,0.987551,-0.006290,0.294958,0.438049,2.939700,6.333333,0.513342,...,1173.0,27.0,1.0,0.199952,0.0,-0.092514,0,0,0,1.559600e+04
5095033,2024-09-30,688800,0.016092,0.765611,0.026757,-0.003641,1.260179,-5.031237,0.234568,0.030825,...,1164.0,36.0,1.0,0.200183,0.0,0.014653,0,0,0,5.167404e+05
5095034,2024-09-30,688819,0.365498,0.780060,0.002957,2.419838,0.949853,3.450847,2.479167,2.604823,...,1349.0,29.0,1.0,0.200000,0.0,-0.005274,0,0,0,1.097981e+06
5095035,2024-09-30,688981,0.238179,0.793408,0.132122,5.980030,1.966938,6.692946,1.431962,6.195305,...,1535.0,28.0,1.0,0.199957,0.0,0.047461,0,0,0,2.000000e+07


In [4]:
temp = total_factor[total_factor['date'] >= pd.to_datetime("2020-07-01")]
del total_factor
total_factor = temp
del temp
gc.collect()
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,2216,2217,2218,2219,2220,label,qcut,weight,qid,amount
424421,2020-07-01,000001,0.032755,0.843080,-0.074178,-3.458246,0.447829,-7.749803,0.534989,-3.410864,...,10682.0,70.0,1.0,0.100000,0.0,0.031135,0,0,0,3718053.00
424422,2020-07-01,000002,0.086861,0.737642,-0.144779,-0.831296,0.356911,-3.153102,0.800866,-0.782118,...,10746.0,63.0,1.0,0.099923,0.0,0.065511,0,0,0,8181773.60
424423,2020-07-01,000004,0.632580,0.284831,-0.511711,-1.308610,0.095793,-4.276766,0.350877,-1.302909,...,10761.0,63.0,0.0,0.100135,0.0,-0.030649,0,0,0,888040.00
424424,2020-07-01,000005,0.000000,0.976718,-0.009933,0.996359,0.529879,-2.759158,0.360000,1.021340,...,10787.0,38.0,0.0,0.100386,0.0,0.003909,0,0,0,18410.00
424425,2020-07-01,000006,0.108898,0.533500,-0.257871,-1.452850,0.265997,2.785404,1.405660,-1.434770,...,10292.0,61.0,1.0,0.100152,0.0,0.099182,0,0,0,1713309.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5095032,2024-09-30,688799,0.022639,0.987551,-0.006290,0.294958,0.438049,2.939700,6.333333,0.513342,...,1173.0,27.0,1.0,0.199952,0.0,-0.092514,0,0,0,15596.00
5095033,2024-09-30,688800,0.016092,0.765611,0.026757,-0.003641,1.260179,-5.031237,0.234568,0.030825,...,1164.0,36.0,1.0,0.200183,0.0,0.014653,0,0,0,516740.40
5095034,2024-09-30,688819,0.365498,0.780060,0.002957,2.419838,0.949853,3.450847,2.479167,2.604823,...,1349.0,29.0,1.0,0.200000,0.0,-0.005274,0,0,0,1097981.04
5095035,2024-09-30,688981,0.238179,0.793408,0.132122,5.980030,1.966938,6.692946,1.431962,6.195305,...,1535.0,28.0,1.0,0.199957,0.0,0.047461,0,0,0,20000000.00


In [ ]:
# 只保留2020-07-01至2024-09-30的数据
total_factor = total_factor.fillna(0)
total_factor.head()

因子筛选辅助数据

In [ ]:
factor_columns = [col for col in total_factor.columns if col not in ['date', 'Code', 'label', 'week_group', 'qcut', 'weight', 'qid', 'amount']]

# 计算因子每天的ic，注释掉的部分可以任意筛掉不同收益率分位数的股票
def compute_all_corrs(df, power=1, target_col='label'):
    df = df.dropna()
    data = df[factor_columns].to_numpy()
    target = df[target_col].to_numpy()
    # 用因子的几次方计算ic
    data = data ** power
    # 中心化数据
    target_centered = target - target.mean()
    data_centered = data - data.mean(axis=0)
    # 协方差
    covariances = np.mean(data_centered * target_centered[:, np.newaxis], axis=0)
    # 标准差
    std_target = target.std()
    std_features = data.std(axis=0)
    std_features[std_features==0] = 1
    # 相关系数
    correlations = covariances / (std_target * std_features)

    del df
    del target
    del data
    del target_centered
    del data_centered
    del std_target
    del std_features
    del covariances
    gc.collect()
    
    return pd.Series(correlations, index=factor_columns)

In [ ]:
# 按天分组计算相关性
correlation_df1 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=1)).reset_index()
correlation_df2 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=2)).reset_index()
correlation_df3 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=3)).reset_index()
correlation_df1.set_index('date', inplace=True)
correlation_df2.set_index('date', inplace=True)
correlation_df3.set_index('date', inplace=True)
correlation_df1 = correlation_df1.abs()
correlation_df2 = correlation_df2.abs()
correlation_df3 = correlation_df3.abs()
correlation_df = pd.DataFrame(
    np.maximum.reduce([correlation_df1.values, correlation_df2.values, correlation_df3.values]),
    columns=correlation_df1.columns, 
    index=correlation_df1.index
    )
correlation_df.to_pickle('/home/datamake117/Documents/haris/dataset/corr_byday_abs.pkl')
correlation_df